In [253]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense


In [256]:
df = pd.read_csv('historical_country_mexico_indicator_gasoline_prices_.csv')
df = df[['DateTime', 'Close']]
df['DateTime'] = pd.to_datetime(df['DateTime'])
df = df.set_index('DateTime')
fecha_inicio = '2020-05-01'
df = df[fecha_inicio:]
df.head()


,Close
DateTime,
2020-05-31,0.87
2020-06-30,0.84
2020-07-31,0.85
2020-08-31,0.86
2020-09-30,0.82


In [257]:
import plotly.graph_objects as go

# Crear la figura
fig = go.Figure()


fig.add_trace(go.Scatter(
    x=df.index,
    y=df['Close'],
    mode='lines',
    name='Precio Histórico',
    line=dict(color='#00CC96', width=2) # Color verde cian
))



# Configuración del diseño (Layout)
fig.update_layout(
    title="Precio de la gasolina en mexico — Histórico",
    xaxis_title="Fecha",
    yaxis_title="Precio (lt/T)",
    template="plotly_dark",      # Fondo oscuro
    hovermode="x unified",       # Muestra el valor al pasar el ratón
    legend=dict(
        orientation="h",         # Leyenda horizontal arriba
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)



In [258]:
# Convertir a numpy para el modelo
serie = df['Close'].values
# Definir tamaño de ventana
window_size = 12

def crear_ventanas(datos, window_size):
    X, y = [], []
    for i in range(len(datos) - window_size):
        X.append(datos[i : i + window_size])
        y.append(datos[i + window_size])
    return np.array(X), np.array(y)

In [259]:
porcentaje_train = 0.8
n_train = int(len(serie) * porcentaje_train)

# Ajustamos tamaño a (n,1) para que el minmax scaler funcione correctamente
serie = serie.reshape(-1, 1)
serie_train = serie[:n_train] # Solo el pasado
serie_test  = serie[n_train:] # Solo el futuro

In [260]:
#  Ajustar el MinMaxScaler SOLO con el entrenamiento
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(serie_train)

# Transformar train y test con el scaler ya entrenado
serie_train_scaled = scaler.transform(serie_train) 
serie_test_scaled  = scaler.transform(serie_test)  

In [261]:
X_train, y_train = crear_ventanas(serie_train_scaled, window_size)
X_test,  y_test  = crear_ventanas(serie_test_scaled,  window_size)

In [262]:
model = Sequential([
    Input(shape=(window_size, 1)),
    LSTM(units=10),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_18 (LSTM)                  │ (None, 10)             │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 491 (1.92 KB)

 Trainable params: 491 (1.92 KB)

 Non-trainable params: 0 (0.00 B)

In [263]:
hist = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - loss: 0.1074 - val_loss: 0.1317
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0932 - val_loss: 0.1098
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0800 - val_loss: 0.0908
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0683 - val_loss: 0.0742
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0578 - val_loss: 0.0594
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0487 - val_loss: 0.0464
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0403 - val_loss: 0.0353
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0329 - val_loss: 0.0257
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0269 - val_loss: 0.0177
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0215 - val_loss: 0.0116
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0173 - val_loss: 0.0070
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0141 - val_loss: 0.0041


In [264]:
# Predicciones en test pero resultado esta en escala {0,1}
y_pred_scaled = model.predict(X_test)

# Convertir a columna (N,1)
y_test_scaled_2d = y_test.reshape(-1,1)
y_pred_scaled_2d = y_pred_scaled.reshape(-1,1)

# Inverse transform
y_test_real = scaler.inverse_transform(y_test_scaled_2d).flatten()
y_pred_real = scaler.inverse_transform(y_pred_scaled_2d).flatten()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


In [265]:
# Ajustar índices de tiempo para alinearlos con la serie original
idx_inicio_test = n_train + window_size
idx_start_test = n_train + window_size
idx_end_test = idx_start_test + len(y_test_real)

fechas_test = df['Close'].index[idx_start_test : idx_end_test]

In [266]:
# Graficar resultados en escala real

fig = go.Figure()

# Serie completa real
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['Close'].values,
    mode="lines",
    name="Serie real completa"
))

# Valores reales del test
fig.add_trace(go.Scatter(
    x=fechas_test,
    y=y_test_real,
    mode="lines",
    name="Valor real (test)"
))

# Predicciones ya invertidas
fig.add_trace(go.Scatter(
    x=fechas_test,
    y=y_pred_real,
    mode="lines",
    name="Predicción LSTM (test)",
    line=dict(dash="dot")
))

fig.update_layout(
    title="Predicción sobre el conjunto de prueba (escala real)",
    xaxis_title="Fecha",
    yaxis_title="Precio de Acción"
)



In [267]:
n_future = 4
future_scaled_preds = []

# Última ventana real, pero en la escala ya transformada
last_window_scaled = serie_train_scaled[-window_size:]  # ventana de train escalada
last_window_scaled = np.concatenate([last_window_scaled, serie_test_scaled])[-window_size:]

# Convertir a shape LSTM: (1, window_size, 1)
current_window = last_window_scaled.reshape(1, window_size, 1)

for _ in range(n_future):
    next_scaled = model.predict(current_window, verbose=0)[0][0]
    future_scaled_preds.append(next_scaled)
    
    # Desplazar ventana
    new_window = np.append(current_window[:,1:,:], [[[next_scaled]]], axis=1)
    current_window = new_window

In [268]:
# Convertir a columna (N,1) para inverse_transform
future_scaled_2d = np.array(future_scaled_preds).reshape(-1, 1)

# Regresar a valores reales sin escala
future_real = scaler.inverse_transform(future_scaled_2d).flatten()

In [270]:
last_date = df['Close'].index[-1]
future_dates = pd.bdate_range(last_date, periods=n_future+1)

In [272]:
fig = go.Figure()

# 1. Serie histórica real
fig.add_trace(go.Scatter(
    x=df['Close'].index,
    y=df['Close'].values,
    mode="lines",
    name="Serie histórica (real)"
))

# 2. Predicción futura (50 días)
fig.add_trace(go.Scatter(
    x=future_dates,
    y=future_real,
    mode="lines+markers",
    name="Pronóstico futuro (50 días)",
    line=dict(dash="dot")
))

fig.update_layout(
    title="Pronóstico de días subsecuentes con LSTM",
    xaxis_title="Fecha",
    yaxis_title="Precio de Acción",
    template="plotly_white"
)

fig.show()

In [ ]:
fig = go.Figure()

# Serie histórica completa (gris claro)
fig.add_trace(go.Scatter(
    x=df['Close'].index,
    y=df['Close'].values,
    mode="lines",
    name="Serie histórica (real)",
    line=dict(color="lightgray")
))

# Serie de entrenamiento (azul)
fig.add_trace(go.Scatter(
    x=df['Close'].index[:n_train],
    y=df['Close'].values[:n_train],
    mode="lines",
    name="Datos de entrenamiento",
    line=dict(color="blue")
))

# Serie de prueba real (naranja)
fig.add_trace(go.Scatter(
    x=fechas_test,
    y=y_test_real,
    mode="lines",
    name="Datos reales (test)",
    line=dict(color="orange")
))

# Predicción del test (verde punteado)
fig.add_trace(go.Scatter(
    x=fechas_test,
    y=y_pred_real,
    mode="lines",
    name="Predicción LSTM (test)",
    line=dict(color="green", dash="dot")
))

# Pronóstico futuro (rojo)
fig.add_trace(go.Scatter(
    x=future_dates,
    y=future_real,
    mode="lines+markers",
    name="Pronóstico futuro (50 días)",
    line=dict(color="red", dash="dot")
))

fig.update_layout(
    title="Resultados y análisis de predicciones con LSTM",
    xaxis_title="Fecha",
    yaxis_title="Precio de Acción",
    template="plotly_white"
)

fig.show()

NameError: name 'close_series' is not defined